# Solving Single Decisions

## The "Party Problem" example, Value of Information

__ A continuation of partyproblem_xdsl.ipynb__

JMA 17 Sept 2025

In [ ]:
# Imports from the python standard library
import math, re, os, sys
from pathlib import Path
import itertools            # to flatten lists

# Import array and dataframe packages
import numpy as np
# import numpy.linalg as la
import pandas as pd

import networkx as nx

# for extract_net
# from ID_operations import * 
from potential_operations import *
import BN

## Bayes networks object

In [ ]:
NETWORK_FILE = 'PartyProblem_asym.xdsl'
# BN structure is contained under the node branch
parsed = BN.extract_net(NETWORK_FILE)
nodes, extensions = parsed
# tags tell the node type. 
# [( k.get('id'), k.tag) for k in nodes]
# CPT contents are stored in row major order (first row, second row, ...)
# Parents are the first matrix dimension -- matrix is Row Markov
pp_net = BN.reap(parsed)
pp_net.pr_nodes()

## Add Weather as a conditioning variable

Modify the decision Potential to include a dimension for the Weather variable

### Solution order of node operations

- Add observation dimension to the Party_location decision
- Compute EV for each option, conditioned on weather state
- Maximize over options (conditioned on weather)
- Update Decn policy with maximum entries (0-1s)
- Expect out weather to get EV

In [ ]:
# Add an additional dimension for a observational arc
# to  extend a decision policy table. 
decn_w_observation = condition_decision(
    pp_net.get_potential('Party_location'), 
    pp_net.get_potential('Weather'))
decn_w_observation

In [ ]:
v,  ix = pp_net.get_potential('Preferences').cpt.max(dim=1)
v, ix # pp_net.get_potential('Preferences').cpt.gather(1, ix)

In [ ]:
# This is how multiple elements can be indexed 
x2 = (tuple(range(2)), ix)
v ==pp_net.get_potential('Preferences').cpt[range(2), list(ix)]

In [ ]:
inx = pp_net.get_potential('Preferences').cpt.argmax(dim=1)
decn_w_observation.cpt[range(2), list(inx)] = 1
decn_w_observation.pr_potential()

### Take expectation of decisions over weather

The expected value if the decision is made __after__ the weather is known

See Figure 12.6, p.252, Howard & Abbas


In [ ]:
# marginalize_utility() 
# TODO use potential_operations
max_decn = (pp_net.get_potential('Preferences').cpt.squeeze(2) * decn_w_observation.cpt).sum(1)
'EV with observation: ', (max_decn * pp_net.get_potential('Weather').cpt).sum().item()